# Question Generation

In [13]:
import logging
import sys
import pandas as pd

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [22]:
from llama_index.evaluation import DatasetGenerator, QueryResponseEvaluator
from llama_index import SimpleDirectoryReader, VectorStoreIndex, ServiceContext, LLMPredictor, Response
from langchain.chat_models import ChatOpenAI

In [3]:
reader = SimpleDirectoryReader("../paul_graham_essay/data")
documents = reader.load_data()

In [4]:
data_generator = DatasetGenerator.from_documents(documents)

Unknown max input size for gpt-3.5-turbo, using defaults.


In [ ]:
eval_questions = data_generator.generate_questions_from_nodes()

In [6]:
eval_questions

['What were the two main things the author worked on before college?',
 'What language did the author use to write programs on the IBM 1401?',
 "What was the author's first microcomputer and what did they use it for?",
 'What did the author study in college before switching to AI?',
 'What made the author want to work on AI?',
 'What language was commonly used for AI in the mid-1980s?',
 'What did the author realize about AI during their first year of grad school?',
 'What did the author decide to focus on instead of AI?',
 'What was the problem with systems work according to the author?',
 'What did the author realize about making art while looking at a painting at the Carnegie Institute?',
 "What was the topic of the author's dissertation?",
 'Which art schools did the author apply to and which one did they end up attending?',
 "What was the author's experience like at the Accademia di Belli Arti in Florence?",
 'What did the author learn about low-end software while working at Inter

In [15]:
# gpt-4
llm_predictor_gpt4 = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-4"))
service_context_gpt4 = ServiceContext.from_defaults(llm_predictor=llm_predictor_gpt4)

In [23]:
evaluator_gpt4 = QueryResponseEvaluator(service_context=service_context_gpt4)

In [9]:
# create vector index
vector_index = VectorStoreIndex.from_documents(
    documents, 
    service_context=service_context_gpt4
)


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 17617 tokens
> [build_index_from_nodes] Total embedding token usage: 17617 tokens


In [28]:
# define jupyter display function
def display_eval_df(query: str, response: Response, eval_result: str) -> None:
    eval_df = pd.DataFrame(
        {
            "Query": query,
            "Response": str(response), 
            "Source": response.source_nodes[0].source_text[:1000] + "...",
            "Evaluation Result": eval_result
        },
        index=[0]
    )
    eval_df = eval_df.style.set_properties(
        **{
            'inline-size': '600px',
            'overflow-wrap': 'break-word',
        }, 
        subset=["Response", "Source"]
    )
    display(eval_df)

In [ ]:
query_engine = vector_index.as_query_engine()
response_vector = query_engine.query(eval_questions[1])
eval_result = evaluator_gpt4.evaluate(eval_questions[1], response_vector)

In [29]:
display_eval_df(eval_questions[1], response_vector, eval_result)

,Query,Response,Source,Evaluation Result
0,What language did the author use to write programs on the IBM 1401?,The author used an early version of Fortran to write programs on the IBM 1401.,"What I Worked On February 2021 Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep. The first programs I tried writing were on the IBM 1401 that our school district used for what was then called ""data processing."" This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights. The language we used was an early version of Fortran. You had to type programs on punch cards, then stack them in...",YES
